In [3]:
# import libraries 

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
%matplotlib inline
import random

In [4]:
# function definitions 

def derive(f,x,h):
    
    return (f(x+h) - f(x-h))/(2*h)

def derive_twice(f,x,h):
    
    return ( f(x+h) - 2*f(x) + f(x-h) ) / (h**2)

def create_interp(x,V):
    
    f = interpolate.interp1d(x,V,kind="cubic",fill_value="extrapolate")
    
    return f

In [45]:
def Verlet (x_data,E_data,i,m,x0,v0,tmax,dt,b):
    
    t = 0; x = x0; v = v0
    
    #V_init = create_interp(x_data,E_data[i]) 
    
    V_set = [elem for elem in E_data] 
    
    pos = []; vel = []; energy = []; time = []; pot = []; 
    
    j = i
    
    L = m * b * v0 # v_inf ?= v0
    
    
    V_current = create_interp(x_data,V_set[j])
    
    a = -derive(V_current,x,dt)/m + (L**2 /(m**2 * x**3)) # additional term coming from the central potential
    
    
    while(t<tmax):                     
            
            V_current = create_interp(x_data,V_set[j])
            
            x_tmp = x
            x = x_tmp + v*dt + 0.5*(dt)**2 * a
            
            a_tmp = a
            a = -derive(V_current,x,dt)/m + (L**2 /(m**2 * x**3))
            
            v_tmp = v
            v = v_tmp + 0.5*(dt)*(a + a_tmp)
            T = 0.5*m*v**2
            
            for k in range(len(V_set)):
                
                if k == j:
                    continue
                
                V_hop = create_interp(x_data,V_set[k])
                
                V_diff = V_current(x) - V_hop(x)  
                
                dV_diff = derive(V_current,x,dt) - derive(V_hop,x,dt) 
                
            
                if dV_diff <= 1e-8: # when the derivative is effectively zero
                    
                    
                    P_LZ = np.exp( (-np.pi/2) * (np.abs(V_diff))**3/2 * (3*np.abs( derive_twice(V_current,t+dt,dt) - derive_twice(V_hop,t+dt,dt) ) )**(-0.5) )              
                    

                    num = random.uniform(0,1)

                    
                    if P_LZ > num:
                        
                            
                        v_aft = np.sqrt(v**2 + (2/m)*V_diff)     # rescale the velocity (?)
                        T_aft = 0.5*v_aft**2

                        if T_aft - (V_diff + T) <= 1e-8: #check the energy gap 
                            
                            
                            j = k
                            T = T_aft
                            
                            print("Hopping to curve: " , k)
                            index = k
                            break
                            
                        
            
            
                
            t = t + dt
            
            pos.append(x)
        
            vel.append(v)
            
            pot.append(V_current(x))
           
            energy.append(V_current(x) + T)
            
            time.append(t)
            
        
    return(index)

In [46]:
# the first column contains the position

x_data = np.loadtxt("myData.txt", delimiter=' ',skiprows=2)[:,0]       

# other columns contain energy values

E_data = [np.loadtxt("myData.txt", delimiter=' ',skiprows=2)[:,i] for i in range(1,8)]

In [48]:
i = int(input("Choose the initial curve (0 for ground state): "))

Choose the initial curve (0 for ground state): 6


In [49]:
m_ccl = 1836.15*(35.453 * 12.0107)/(35.453 + 12.0107)
E = float(input("Initial kinetic energy: "))
b = float(input("Impact parameter: "))
v_ccl = (2*E/m_ccl)**0.5
x0 = -90

Initial kinetic energy: 1e3
Impact parameter: 0.2


In [50]:
Verlet(x_data,E_data,i,m_ccl,x0,v_ccl,180/v_ccl,0.01,b)

Hopping to curve:  5


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in double_scalars


Hopping to curve:  6
Hopping to curve:  5
Hopping to curve:  4
Hopping to curve:  6
Hopping to curve:  5
Hopping to curve:  4
Hopping to curve:  5
Hopping to curve:  4
Hopping to curve:  6
Hopping to curve:  5
Hopping to curve:  4
Hopping to curve:  5
Hopping to curve:  4
Hopping to curve:  6


6

In [ ]:
# Return the index of the last state for 100 trajectories

for elem in range(100):
    ind = []
    ind.append(Verlet(x_data,E_data,i,m_ccl,x0,v_ccl,180/v_ccl,0.01,b))

In [ ]:
# Probability of hopping to each state

P_i = []
for num in range(7):
    P_i.append(ind.count(num) / len(ind))